В предложенном датасете voiceDataSet.csv есть ряд свойств, определенных по аудиозаписям: 

- meanfreq: средняя частота голоса (в кГц)
- sd: стандартное отклонение частоты голоса
- median: медианная частота (в кГц)
- Q25: значение в первом квартиле (в кГц)
- Q75: значение в третьем квартиле (в кГц)
- IQR: интерквартильный размах (в кГц)
- skew: ассиметрия
- kurt: эксцесс
- sp.ent: спектральная энтропия
- sfm: энтропия Винера
- mode: мода частоты
- centroid: частотный центроид
- meanfun: средняя основная частота, измеренная по акустическому сигналу
- minfun:  минимальная основная частота, измеренная по акустическому сигналу
- maxfun: максимальная основная частота, измеренная в акустическом сигнале
- meandom: среднее значение доминирующей частоты, измеренной по акустическому сигналу
- mindom: минимум доминирующей частоты, измеренной в акустическом сигнале
- maxdom: максимум доминирующей частоты, измеренной в акустическом сигнале
- dfrange: диапазон доминантных частот, измеренное на звуковой сигнал
- modindx: индекс модуляции голоса

## Задание

Попробуйте построить модель, предсказывающую пол обладателя записи голоса.

Для этого:

Разделите выборку на обучающую и тренировочную с параметрами test_size=0.3, random_state=42.

Нормализуйте признаки с помощью функции StandardScaler(). Учитывайте, что нормализация тестовой выборки производится по среднему и отклонению тренировочной, которую мы считаем репрезентативной относительно генеральной совокупности.

Обучите модель логистической регрессии на подготовленных данных.

Пояснение: 

Тестовые данные не должны влиять на параметры нормализации. Нужно использовать SCALER.TRANSFORM вместо SCALER.FIT_TRANSFORM, чтобы применять параметры нормализации, рассчитанные для тренировочных данных. Иначе данные в трейне и в тесте будут нормализованы по - разному.

In [25]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv('../data/voiceDataSet.csv')

In [9]:
data.loc[:, 'label'] = data['label'].apply(lambda x: 1 if x == 'male' else 0)

In [14]:
X = data.drop(columns=['label']).values
y = data['label'].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
scaler = StandardScaler()
scaler.fit(X_train, y_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [24]:
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

In [26]:
model = LogisticRegression()
model.fit(X_train_sc, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
y_predict = model.predict(X_test_sc)

MSE = mean_squared_error(y_test, y_predict)
display(MSE)

0.027339642481598318

## Задание 4.7.1

Вычислите значение метрики Accuracy и введите полученное значение. Запишите ответ, округлив его до третьего знака после запятой.

In [28]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [32]:
round(accuracy_score(y_test, y_predict), 3)

0.973

__________________________________________
Теперь познакомимся с новым алгоритмом классификации, а также снова потренируемся делить выборку и осуществлять кросс-валидацию. Кроме того, посмотрим, как можно оценить качество классификации для случая, когда наша классификация не бинарная, т.е. у нас несколько классов.

В этом кейсе мы попробуем определять типы стекла по его характеристикам.

In [33]:
import pandas as pd

In [34]:
data = pd.read_csv('../data/glass.csv')

Откроем данные и увидим, что в первых столбцах показатели содержания различных веществ в стекле, а в последнем (Type) — непосредственно тип стекла.

Наша классификация мультиклассовая, поэтому мы должны посмотреть, сколько у нас классов. Если вы вообще ничего не знаете о задаче в плане количества классов, обязательно начинайте с этого, поскольку это повлияет на ход решения. Особенно важно понять, бинарная классификация или нет.

In [37]:
data.head(3)

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


In [41]:
X = data.drop(columns=['Type']).values
y = data['Type'].values

## Задание 4.7.2

Сколько классов стекла представлено в этой задаче?

In [40]:
data['Type'].nunique()

6

Итак, мы выяснили, сколько у нас классов и узнали, что в этот раз у нас будет не бинарная классификация. А значит, мы не сможем использовать некоторые метрики качества (например, precision и recall).

Приступим непосредственно к построению модели. На примере этой задачи мы узнаем новый алгоритм **kNN (метод ближайших соседей)**. Это один из простейших методов классификации. 

Его называют **ленивым классификатором**, потому что во время обучения модели он ничего не делает, просто считывает и сохраняет тренировочные данные. Сама классификация для него начинается тогда, когда ему дают тестовые данные. Тогда kNN проходит два базовых шага:

 1. Сначала он ищет k ближайших размеченных точек данных – эти точки и называют **k ближайшими соседями**.
 2. Затем, используя классы соседей, kNN решает, как лучше классифицировать новые данные. 

Пример

Предположим, что алгоритм хочет классифицировать вас по профессии. Он видит, что люди, рядом с которыми вы находитесь (родители, друзья), — врачи. Тогда он решает, что вы — тоже скорее всего врач. Грубо говоря, он основывается на принципе, что подобное находится рядом с подобным.

А что делать, если ближайшие соседи не принадлежат одному классу? Можно решать по принципу большинства или дать каждому соседу вес (в зависимости от удаленности), с которым он будет вносить вклад в решение.

Например, попробуем классифицировать зеленый кружочек. На этом примере мы видим, что выбор числа соседей (это настраиваемый параметр) очень важен.

<img src="../images/KNN_classify.png" alt="precision-recall-curve" width="400" align="center"> 

Если мы будем рассматривать трех соседей, то кружочек будет причислен к классу треугольников. А если пять соседей — к классу квадратов.

**Важно! Количество соседей должно быть нечетное во избежание спорной ситуации**.

Итак, мы разобрались с новым алгоритмом. Теперь воспользуемся k-fold валидацией на пяти разбиениях и обучим модель:

In [42]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold

model = KNeighborsClassifier(n_neighbors=3)
kf = KFold(n_splits=5)
cross_val_score(model, X, y, cv=kf, scoring="accuracy")

array([0.44186047, 0.65116279, 0.3255814 , 0.34883721, 0.07142857])

## Задание 4.7.3

Измените количество разбиений на 10. Вычислите среднее значение метрики Accuracy по 10 разбиениям и введите ниже, округлите до сотых.

In [44]:
kf = KFold(n_splits=10)
acc = cross_val_score(model, X, y, cv=kf, scoring="accuracy")

In [46]:
acc.mean()

0.5257575757575758

__________

В следующей задаче мы будем диагностировать болезни сердца по различным медицинским параметрам пациентов.

In [48]:
import pandas as pd

In [53]:
data = pd.read_csv('../data/heart_fin1.csv', sep=';')

In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
age         303 non-null int64
sex         303 non-null int64
cp          303 non-null int64
trestbps    303 non-null int64
chol        303 non-null int64
restecg     303 non-null int64
thalach     303 non-null int64
exang       303 non-null int64
oldpeak     303 non-null float64
target      303 non-null int64
dtypes: float64(1), int64(9)
memory usage: 23.8 KB


Избавьтесь от выбросов, чтобы они не ухудшили качество нашей модели. Для того, чтобы избавиться от выбросов, уберите все данные, значения признаков которых отстают от первого или третьего квартиля более, чем на 1.5 межквартильного размаха.

In [56]:
def remove_outliers_iqr(data: pd.DataFrame, column: str) -> pd.DataFrame: 
    '''Функция для удаления выбросов с помощью IQR. 
    data - набор исходных данных
    column - наименование признака
    
    Функция производит расчет граница выбросов с помощью IQR.
    Удаляет строки исходного набора данных (data), для которых значения признака (column) 
    не попадают в границаы выбросов. Пустые значения оставляет.
    Функция возвращает скорректированный набор данных.
    
    '''
    
    perc25 = data[column].quantile(0.25)
    perc75 = data[column].quantile(0.75)
    IQR = perc75 - perc25
    lower_limt = perc25 - 1.5*IQR
    upper_limit = perc75 + 1.5*IQR  
    
    return data.query(f'{lower_limt} <= {column} <= {upper_limit} or {column} != {column}')

In [60]:
columns = data.columns[: -1]
columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'restecg', 'thalach', 'exang',
       'oldpeak'],
      dtype='object')

In [61]:
for column in columns:
    data = remove_outliers_iqr(data, column)

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 302
Data columns (total 10 columns):
age         284 non-null int64
sex         284 non-null int64
cp          284 non-null int64
trestbps    284 non-null int64
chol        284 non-null int64
restecg     284 non-null int64
thalach     284 non-null int64
exang       284 non-null int64
oldpeak     284 non-null float64
target      284 non-null int64
dtypes: float64(1), int64(9)
memory usage: 24.4 KB


## Задание 4.7.4

Сколько наблюдений осталось после удаления выбросов?

In [63]:
data.shape

(284, 10)

## Задание 4.7.5-4.7.6

Теперь разбейте выборку на тестовую и обучающую с параметрами test_size=0.15, random_state=5.

Обучите модели логистической регрессии ( c параметром max_iter=1000) и KNN (с количеством соседей, равным 3) на этих данных. Вычислите метрики качества.

У какой модели выше значение ROC AUC?

In [94]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [95]:
data = pd.read_csv('../data/heart_fin1.csv', sep=';')

for column in data.columns[: -1]:
    data = remove_outliers_iqr(data, column)

In [96]:
X = data.drop(columns=['target']).values
y = data['target']

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=5)

In [101]:
model_1 = LogisticRegression(solver='lbfgs', max_iter=1000)
model_1.fit(X_train, y_train)

y_pred_1 = model_1.predict_proba(X_test)
roc_auc_1 = roc_auc_score(y_test, y_pred_1[:, 1])
display(roc_auc_1)

0.8181818181818181

In [99]:
model_2 = KNeighborsClassifier(n_neighbors=3)
model_2.fit(X_train, y_train)

y_pred_2 = model_2.predict_proba(X_test)
roc_auc_2 = roc_auc_score(y_test, y_pred_2[:, 1])
display(roc_auc_2)

0.6547619047619049

In [102]:
print(f'LogisticRegression ROC_AUC = {roc_auc_1} KNeighborsClassifier ROC_AUC = {roc_auc_2}')

LogisticRegression ROC_AUC = 0.8181818181818181 KNeighborsClassifier ROC_AUC = 0.6547619047619049
